***Importing Required Libraries***

In [ ]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

import re
import time
import nltk
import praw
import string
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import gensim
from gensim import corpora
from gensim.models import Word2Vec, CoherenceModel


from sklearn.model_selection import train_test_split

import nltk
from nltk.collocations import *
from nltk import FreqDist, word_tokenize
from nltk.corpus import gutenberg, stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from keras.preprocessing import text
from keras.models import Model, model_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import History, EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Acquiring User Likes


## Using Twitter API

In [ ]:
def get_liked_tweets(handle):
    '''
    Takes Twitter handle and extracts and parses liked tweets.
    
    '''
    ### Acquired from Twitter using API key and API secret key ###
    bearer_token = 'AAAAAAAAAAAAAAAAAAAAAImyAwEAAAAAyWB9oAfMqhJG66%2BeJHuUEi5wsXM%3DKOzTDPgjqoXjzERO1JmoAJfVIqjnaPdD7wUhayNjESzeytty2J'
    ###
    all_tweets = []
    url = 'https://api.twitter.com/1.1/favorites/list.json?screen_name={}'.format(handle)
    headers = {
        'Authorization': 'Bearer {}'.format(bearer_token),
            }
    first_url_params = {
        'count': 200
            }
    first_response = requests.get(url, headers=headers, params=first_url_params)
    print(first_response)
    first_initial_split = first_response.text.split(',"text":"')
    first_t = len(first_initial_split)    
    for i in range(1,first_t):
        all_tweets.append(first_initial_split[i].split(',"truncated":')[0])
        
    ### Regex to extract the id of the last tweet to use as pagination ###      
    regsearch = re.search('(?<=(\d","id":))[\d]*', first_initial_split[first_t-2])
    tweet_id = int(regsearch.group(0))
    ###
    
    ### Looping process to manually paginate through tweets ###
    page = 0
    while page < 15:
        url1 = 'https://api.twitter.com/1.1/favorites/list.json?screen_name={}&count=200&max_id={}'.format(handle,tweet_id-1)
        response = requests.get(url1, headers=headers)
        initial_split = response.text.split(',"text":"')
        t = len(initial_split)
        for i in range(1,t):
            all_tweets.append(initial_split[i].split(',"truncated":')[0])
 
        ### Regex to extract the id of the last tweet to use as pagination ###      
        try:
            regsearch = re.search('(?<=(\d","id":))[\d]*', initial_split[t-2])
            tweet_id = int(regsearch.group(0))
        except: break
        ###
        
        page +=1
    return all_tweets

# Getting Top Subreddits

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.reddit.com/r/TheoryOfReddit/comments/1f7hqc/the_200_most_active_subreddits_categorized_by/')

In [ ]:
subs = driver.find_elements_by_class_name('_1qeIAgB0cPwnLhDF9XSiJM')
subreddits = []
for i in subs[3:]:
    if "r/" in i.text:
        subreddits.append(i.text.split(" - ")[1])
    else:
        continue
### THIS CELL THROWS AN ERROR -- IGNORE IT -- ###

In [ ]:
clean_subs = []
for sub in subreddits:
    clean_subs.append(sub.split('r/')[1])

## Using PRAW (Python Reddit API Wrapper) to get top posts in top subreddits

In [ ]:
reddit = praw.Reddit(client_id='MU9yce7zmiOKwA',
                     client_secret='DRV5fO7p-Tn7VzxmMH8mnRwM_hE',
                     user_agent='my user agent')

In [ ]:
all_submissions = {}
for sub in tqdm(clean_subs):
    all_submissions[sub] = []
    for submission in reddit.subreddit(sub).top(limit=1000):
        all_submissions[sub].append(submission.title)

## Looking at examples of pre-processing and EDA

In [ ]:
lemmatizer = WordNetLemmatizer()
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
subr_text_tokens_raw = nltk.regexp_tokenize(" ".join(all_submissions['creepy']), pattern)
subr_text_tokens = [word.lower() for word in subr_text_tokens_raw]
subr_text_tokens = [lemmatizer.lemmatize(i) for i in subr_text_tokens]
subr_text_freqdist = FreqDist(subr_text_tokens)
subr_text_freqdist.most_common(200)

In [ ]:
subr_text_stopped_freqdist = FreqDist(subr_text_stopped)
subr_text_stopped_freqdist.most_common(50)

In [ ]:
total_word_count = sum(subr_text_stopped_freqdist.values())
subr_text_top_50 = subr_text_stopped_freqdist.most_common(50)
print("Word\t\t\tNormalized Frequency")
for word in subr_text_top_50:
    normalized_frequency = word[1] / total_word_count
    print("{} \t\t\t {:.4}".format(word[0], normalized_frequency))

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

subr_text_finder = BigramCollocationFinder.from_words(subr_text_stopped)
subr_text_finder.apply_freq_filter(3)

subr_text_scored = subr_text_finder.score_ngrams(bigram_measures.pmi)
subr_text_scored[:20]

# Separating subreddits into respective categories

In [ ]:
label_text_df = pd.DataFrame()
label_text_df['target'] = ['Discussion and Stories','Emotional Reaction Fuel','Entertainment - Gaming', 
                         'Entertainment - Television', 'Entertainment - Other (Movies/Music/Franchies/Misc)',
                         'Humor','Images, Gifs, and Videos','Learning and Thinking','Lifestyle and Help',
                         'News and Issues', 'Places', 'Race, Gender, and Identity', 'Sports','Technology']
categorized_posts = dict((i,"")for i in label_text_df.target)
for i in clean_subs:
    if i =="AskReddit" or i=='IAmA' or i =="bestof" or i=='fatpeoplestories' or i =='pettyrevenge' or i =="TalesFromRetail" or i=='DoesAnybodyElse' or i =='CrazyIdeas':
        categorized_posts['Discussion and Stories'] = [categorized_posts['Discussion and Stories'],all_submissions[i]]
    elif i =="WTF" or i=='aww' or i =="cringepics" or i=='JusticePorn' or i =='MorbidReality' or i =="rage" or i=='mildlyinfuriating' or i =='creepy' or i=='creepyPMs' or i =="nosleep" or i=='nostalgia':
        categorized_posts['Emotional Reaction Fuel'] = [categorized_posts['Emotional Reaction Fuel'],all_submissions[i]]
    elif i =="gaming" or i=='leagueoflegends' or i =="pokemon" or i=='Minecraft' or i =='starcraft' or i =="Games" or i=='DotA2' or i =='skyrim' or i=='tf2' or i =="magicTCG" or i=='wow' or i =="KerbalSpaceProgram" or i=='mindcrack' or i =='Fallout' or i=='roosterteeth' or i =="Planetside" or i=='gamegrumps' or i =="battlefield3" or i=='zelda' or i =='darksouls' or i=='masseffect':
        categorized_posts['Entertainment - Gaming'] = [categorized_posts['Entertainment - Gaming'],all_submissions[i]]
    elif i =="arresteddevelopment" or i=='gameofthrones' or i =="doctorwho" or i=='mylittlepony' or i =='community' or i =="breakingbad" or i=='adventuretime' or i =='startrek' or i=='TheSimpsons' or i =="futurama" or i=='HIMYM' or i=='DunderMifflin' or i =="thewalkingdead":
        categorized_posts['Entertainment - Television'] = [categorized_posts['Entertainment - Television'],all_submissions[i]]
    elif i =="Music" or i=='movies' or i =="harrypotter" or i=='StarWars' or i =='DaftPunk' or i =="hiphopheads" or i=='anime' or i =='comicbooks' or i=='geek' or i =="batman" or i=='TheLastAirbender' or i=='Naruto' or i =="FanTheories":
        categorized_posts['Entertainment - Other (Movies/Music/Franchies/Misc)'] = [categorized_posts['Entertainment - Other (Movies/Music/Franchies/Misc)'],all_submissions[i]]
    elif i =="funny" or i=='AdviceAnimals' or i =="fffffffuuuuuuuuuuuu" or i=='4chan' or i =='ImGoingToHellForThis' or i =="firstworldanarchists" or i=='circlejerk' or i =='MURICA' or i=='facepalm' or i =="Jokes" or i=='wheredidthesodago' or i=='polandball' or i =="TrollXChromosomes" or i =="comics" or i=='nottheonion' or i=='britishproblems' or i =="TumblrInAction" or i =="onetruegod":
        categorized_posts['Humor'] = [categorized_posts['Humor'],all_submissions[i]]
    elif i =="pics" or i=='videos' or i =="gifs" or i=='reactiongifs' or i =='mildlyinteresting' or i =="woahdude" or i=='FiftyFifty' or i =='FoodPorn' or i=='HistoryPorn' or i =="wallpapers" or i=='youtubehaiku' or i=='Unexpected' or i =="photoshopbattles" or i =="AnimalsBeingJerks" or i=='cosplay' or i=='EarthPorn' or i =="QuotesPorn" or i =="awwnime" or i =="AbandonedPorn" or i=='carporn' or i =="PerfectTiming" or i=='OldSchoolCool' or i =='RoomPorn' or i =="woahdude" or i=='Pareidolia' or i =='MapPorn' or i=='tumblr' or i =="techsupportgore" or i=='PrettyGirls' or i=='itookapicture':
        categorized_posts['Images, Gifs, and Videos'] = [categorized_posts['Images, Gifs, and Videos'],all_submissions[i]]
    elif i =="todayilearned" or i=='science' or i =="askscience" or i=='space' or i =='AskHistorians' or i =="YouShouldKnow" or i=='explainlikeimfive':
        categorized_posts['Learning and Thinking'] = [categorized_posts['Learning and Thinking'],all_submissions[i]]
    elif i =="trees" or i=='MakeupAddiction' or i =="cats" or i=='LifeProTips' or i =='RedditLaqueristas' or i =="Random_Acts_Of_Amazon" or i=='food' or i =='guns' or i=='tattoos' or i =="corgi" or i=='teenagers' or i =='GetMotivated' or i=='motorcycles' or i =="sex" or i=='progresspics' or i =='DIY' or i=='bicycling' or i =="Fitness" or i=='lifehacks' or i =='longboarding' or i=='Frugal' or i =="drunk" or i=='Art' or i=='loseit' or i =='Military':
        categorized_posts['Lifestyle and Help'] = [categorized_posts['Lifestyle and Help'],all_submissions[i]]
    elif i =="politics" or i=='worldnews' or i =="news" or i=='conspiracy' or i =='Libertarian' or i =="TrueReddit" or i=='Conservative' or i=='offbeat':
        categorized_posts['News and Issues'] = [categorized_posts['News and Issues'],all_submissions[i]]
    elif i =="canada" or i=='toronto' or i =="australia" or i=='unitedkingdom':
        categorized_posts['Places'] = [categorized_posts['Places'],all_submissions[i]]
    elif i =="atheism" or i=='TwoXChromosomes' or i =="MensRights" or i=='gaybros' or i =='lgbt':
        categorized_posts['Race, Gender, and Identity'] = [categorized_posts['Race, Gender, and Identity'],all_submissions[i]]
    elif i =="nba" or i=='soccer' or i =="hockey" or i=='nfl' or i =='formula1' or i =="baseball" or i=='MMA' or i=='SquaredCircle':
        categorized_posts['Sports'] = [categorized_posts['Sports'],all_submissions[i]]
    elif i =="technology" or i=='Android' or i =="Bitcoin" or i=='programming' or i =='apple':
        categorized_posts['Technology'] = [categorized_posts['Technology'],all_submissions[i]]

## Creating dataframe of posts and categories

In [ ]:
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i
posts_by_category = dict((i,"")for i in label_text_df.target)
for i in label_text_df.target:
    posts_by_category[i] = list(flatten(categorized_posts[i]))[1:]

In [ ]:
master_list = []
for i in tqdm(label_text_df.target):
    for post in posts_by_category[i]:
        post_list = []
        post_list.append(i)
        post_list.append(post)
        master_list.append(post_list)

reddit_df = pd.DataFrame(master_list)
reddit_df.to_csv('reddit_df') ### Saving dataframe

In [ ]:
reddit_df = pd.read_csv('reddit_df') ### Loading dataframe
reddit_df = reddit_df.drop(columns=["Unnamed: 0"])
reddit_df.columns = reddit_df.columns.astype("int")

In [ ]:
dictionary = corpora.Dictionary(reddit_df_stopped['tokens'])
dictionary.filter_extremes(no_below=10, no_above=0.66, keep_n=20000)

In [ ]:
def turn2numbers(tokens):
    word_tokens = [dictionary.token2id[token] for token in tokens if token in dictionary.token2id]
    return word_tokens

In [ ]:
reddit_df_stopped['filtered'] = reddit_df_stopped['tokens'].apply(lambda x: turn2numbers(x))

# Using GloVes multi-dimensional word vectorizer to build baseline models

In [ ]:
target = reddit_df[0]
data = reddit_df[1].map(word_tokenize).values

total_vocabulary = set(word for headline in data for word in headline)
print("There are {} unique tokens in the dataset.".format(len(total_vocabulary)))

In [ ]:
glove = {}
with open('glove.6B.300d.txt', 'rb') as f:
    for line in f:                                    
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [ ]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
            
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [ ]:
rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in tqdm(models)]
scores

**Initial results seem low, but, with 14 categories, random guessing would achieve an accuracy of 7.1%. Let's try a Deep Neural Network with a Word Embedding Layer**

In [ ]:
y = pd.get_dummies(target).values
reddit_df['splitted'] = reddit_df[1].apply(lambda x: x.split())

In [ ]:
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]
reddit_df['text_lemmatized'] = reddit_df.splitted.apply(lemmatize_text)

In [ ]:
reddit_df[1] = reddit_df.text_lemmatized.apply(lambda x: " ".join(x))

In [ ]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(reddit_df[1]))
list_tokenized_headlines = tokenizer.texts_to_sequences(reddit_df[1])
X_t = pad_sequences(list_tokenized_headlines, maxlen=100)

In [ ]:
#########################
#      BEST  MODEL      #  --- Epoch 1: 15.6% accurate
#########################  --- Epoch 50: 49.4%

embedding_size = 300
input_ = Input(shape=(50,))
x = Embedding(20000, embedding_size)(input_)
x = Dense(750, activation='relu', kernel_regularizer=regularizers.l2(0.2))(x)

x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(500, activation='relu')(x)
x = Dropout(0.2)(x)
# There are 14 different possible classes, so we use 14 neurons in our output layer
x = Dense(14, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])
X_train,X_val, y_train,y_val = train_test_split(X_t,y,test_size=0.1,random_state=1)
model.summary()

**Neural Network initially kept having a val_accuracy of 0... because the data is sitting in order and wasn't shuffled! (the "test_size" is just the very last bit (0.2) of the data, it doesn't choose randomly... fixed with train test split, which shuffles by default**

# Failed - Training Neural Network with stopwords removed from post titles

In [ ]:
def lemmatize_speech_text(text):
    return [w.lower() for w in nltk.word_tokenize(text)]
reddit_df_stopped['tokens'] = reddit_df_stopped['full_text'].apply(lambda x: lemmatize_speech_text(x))

In [ ]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))

reddit_df_stopped = pd.DataFrame()
reddit_df_stopped[0] = reddit_df[0]
reddit_df_stopped[1] = reddit_df[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
reddit_df_stopped['full_text'] = reddit_df_stopped[1].apply(lambda x: re.sub(r'[^\w\s]', '', x))
reddit_df_stopped['full_text'] = reddit_df_stopped['full_text'].apply(lambda x: re.sub(r'[\d]', '', x))

X_t = pad_sequences(reddit_df_stopped['filtered'], maxlen=50)

In [ ]:
embedding_size = 300
input_ = Input(shape=(50,))
x = Embedding(20000, embedding_size)(input_)
x = Dense(750, activation='relu', kernel_regularizer=regularizers.l2(0.2))(x)

x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dense(500, activation='relu')(x)
x = Dropout(0.2)(x)
# There are 14 different possible classes, so we use 14 neurons in our output layer
x = Dense(14, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])
X_train,X_val, y_train,y_val = train_test_split(X_t,y,test_size=0.3,random_state=1)
model.summary()

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=100, batch_size=500, callbacks=[earlyStopping,mcp_save], shuffle=True, validation_data=(X_val,y_val))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

# Success - Doc2Vec Model

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
data = reddit_df_stopped.full_text
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [ ]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

# Calculating probability of what category you like based on tweets!

In [ ]:
def RedditCategoryPredictor():
    twitter_handle = input("What is your Twitter Handle?")
    likes = get_liked_tweets(twitter_handle)
    print("Grabbed {} likes".format(len(likes)))     
    
    joined_words = []
    for i in set(likes):
        joined_words.append("".join(i))
    posts_df = pd.DataFrame(joined_words)
    posts_df['full_text'] = posts_df[0].apply(lambda x: re.sub(r'[^\w\s]', '', x))         ### Removing Unnecessary
    posts_df['full_text'] = posts_df['full_text'].apply(lambda x: re.sub(r'[\d]', '', x))  ### Characters 
    posts_df['final'] = posts_df['full_text'].apply(lambda x: x.split())

    dictionary = corpora.Dictionary(posts_df['final'])
    dictionary.filter_extremes(no_below=10, no_above=0.66, keep_n=20000)    
    tokens =list(flatten(posts_df['final']))
    new_vector = model.infer_vector(tokens)
    sims = model.docvecs.most_similar([new_vector],topn=50)
    final_results = []
    for i in sims:
        final_results.append(reddit_df_stopped.iloc[int(i[0])][0])
    cnt = Counter()
    for i in final_results:
        cnt[i] += 1
    print(max(cnt))
    return max(cnt)

# LDA Model

In [ ]:
texts = list(reddit_df_stopped.tokens)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=12, random_state=42,
                                             workers=3, passes=10)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Getting Related Subreddits

## Acquiring subreddit posts

In [ ]:
related_subreddits = {}
for i in clean_subs:
    searchbox = driver.find_element_by_id('search-input')
    searchbox.send_keys(i)
    driver.find_element_by_class_name('btn').click()
    result_subs = driver.find_elements_by_xpath("//div[contains(@ng-repeat, 'sub in to')]")
    for s in result_subs:
        related_subreddits.setdefault(i,[]).append(s.text) 
    driver.find_element_by_id('search-input').clear()
        

In [ ]:
driver = webdriver.Chrome()
for related_subs in tqdm(list(related_subreddits.values())):
    for sub in related_subs:
        driver.get('https://www.reddit.com/r/{}'.format(sub))
        try:
            driver.find_element_by_class_name('_1jefpljVGT-eHObg40F8Dm')
            related_subs.remove(sub)
        except:
            continue

In [ ]:
all_related_posts = {}
for related_subs_key,related_subs_val in tqdm(list(related_subreddits.items())):
    related_sub_posts = []
    for sub in tqdm(related_subs_val):
        new_str = ""
        try:
            for submission in reddit.subreddit(sub).top(limit=1000):
                new_str = new_str + submission.title + " "
        except:
            continue
        related_sub_posts.append((new_str,sub,related_subs_key))
    all_related_posts[related_subs_key] = related_sub_posts

In [ ]:
related_subdf = {}
for parent_sub in tqdm(list(related_subreddits.keys())):
    for sub_tuple in all_related_posts[parent_sub]:
        related_subdf[sub_tuple[1]] = sub_tuple[0]

In [ ]:
related_subdf = pd.DataFrame.from_dict(related_subdf,orient='index')

In [ ]:
related_subdf.to_csv('related_subdf')

In [ ]:
related_subdf=pd.read_csv('related_subdf',index_col='Unnamed: 0')
related_subdf.columns=related_subdf.columns.astype('int')

In [ ]:
related_subdf['clean_text'] = related_subdf[0].apply(lambda x: re.sub(r'[^\w\s]', '', x))
related_subdf['clean_text'] = related_subdf['clean_text'].apply(lambda x: re.sub(r'[\d]', '', x))

In [ ]:
data2 = related_subdf.clean_text
tagged_data2 = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data2)]

## Training model on data from related subreddits

In [ ]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model_relatedsubs = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model_relatedsubs.build_vocab(tagged_data2)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model_relatedsubs.train(tagged_data2,
                total_examples=model_relatedsubs.corpus_count,
                epochs=model_relatedsubs.iter)
    # decrease the learning rate
    model_relatedsubs.alpha -= 0.0002
    # fix the learning rate, no decay
    model_relatedsubs.min_alpha = model_relatedsubs.alpha

model_relatedsubs.save("d2v_relatedsubs.model")
print("Model Saved")

In [ ]:
indexed_subs_df = related_subdf.reset_index()

In [ ]:
indexed_subs_df = indexed_subs_df.reset_index()
indexed_subs_df.columns=['index_no','sub','0','clean_text']

In [ ]:
indexed_subs_df = indexed_subs_df.drop(columns=['0'])
indexed_subs_df.to_csv('related_subs_and_indexes')

## Finding most similar subreddits

In [ ]:
def get_most_similar_subreddit(sub_name):
    sub_num = indexed_subs_df[indexed_subs_df['sub']==sub_name]['index_no'].values[0]
    similar_subs = pd.DataFrame(model_relatedsubs.docvecs.most_similar(sub_num, topn=5))
    similar_subs.columns = ['index_no', 'Percent Similarity']
    similar_subs['index_no'] = similar_subs['index_no'].apply(lambda x: int(x))
    similar_subs['Percent Similarity'] = similar_subs['Percent Similarity'].apply(lambda x: str(round(x*100))+'%')
    similar_df = indexed_subs_df.merge(similar_subs, how='inner', on='index_no')[['sub','Percent Similarity']]
    return similar_df.sort_values(by='Percent Similarity',ascending=False)

In [ ]:
def create_speech_wordcloud(row):
    mycmap = 'magma'
    wordcloud = WordCloud(width=1600, height=800,# stopwords=stopwords_list,
                        background_color='white', max_words=120,
                        colormap=mycmap).generate(row['clean_text'])
    filename = 'wordcloud' + str(row['index_no']) + '.png'
    wordcloud.to_file(filename)

In [ ]:
indexed_subs_df.apply(lambda x: create_speech_wordcloud(x), axis=1)